# Investments Project (Spring 2024)

**Authors:**
- Marc-Antoine Allard
- Adam Zinebi
- Paul Teiletche
- ...

**DUE Date: June 21 at 23:59**

---
# Utils

## Imports

In [2]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import wrds
import seaborn as sns

In [9]:
# WRDS connection Set up

# Your Username here
username = "maallard"

db=wrds.Connection(wrds_username=username)
db.create_pgpass_file()

Loading library list...
Done


## Helper Functions

---
# The Data

**Download:**
- Monthly stock returns from CRSP from January 1, 1964, to December 31, 2023, for all common stocks traded on the NYSE and AMEX.
- The value-weighted CRSP market return and 1-month T-bill returns as a risk-free rate.

In [16]:
### Download and format data

#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
Rf=db.raw_sql("""select  mcaldt,tmytm 
            from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
            and mcaldt>='1964-01-01'
            and mcaldt<='2023-12-31'""", date_cols=['mcaldt'])
Rf['tmytm']=Rf['tmytm']/12/100
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#---------------------------------------------
# Value Weighted Index Returns
#---------------------------------------------
Rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='1964-01-01' and date<='2023-12-31'
                """,date_cols=['date'])
Rm = Rm.rename(columns={'vwretd':'Rm'})

#---------------------------------------------
# Stock Returns 
#---------------------------------------------

stock_data = db.raw_sql("""
                    select a.permno, a.date, a.ret, a.shrout, a.prc, b.siccd 
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1964' and '12/31/2023'
                      and b.exchcd between 1 and 2 
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 
stock_data = stock_data.rename(columns={'ret':'Rn'})


#---------------------------------------------
# Merge Data
#---------------------------------------------

# Merge all data
merged_data = pd.merge(stock_data, Rm, on='date')
merged_data = pd.merge(merged_data, Rf, on='date')

# Save CSV
merged_data.to_csv('merged_stock_data.csv', index=False)

## Load and Prepare DATA

In [18]:
data = pd.read_csv("../data/merged_stock_data.csv")

In [19]:
# CLEAN 
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(['permno', 'date']) # Sort the data by 'permno' and 'date'
data['mcap'] = np.abs(data['prc']) * data['shrout'] # Market capitalization
data['mcap_l'] = data.groupby(['permno'])['mcap'].shift(1) # Lagged market capitalization
data['Rn_f'] = data.groupby(['permno'])['Rn'].shift(-1) # Future stock return
# CST col
data['const'] = 1
# Excess returns
data['Rn_e'] = data['Rn'] - data['rf']
data['Rm_e'] = data['Rm'] - data['rf']
# Market capitalization weight
data['w_m'] = data['mcap_l'] / data.groupby(['date'])['mcap_l'].transform('sum')

# Drop rows with missing values in specified columns
data = data.dropna(subset=['mcap_l', 'Rn_e', 'Rm_e']).copy()

# Display the prepared DataFrame
data.head()

,permno,date,Rn,shrout,prc,siccd,Rm,rf,mcap,mcap_l,Rn_f,const,Rn_e,Rm_e,w_m
5,10001,2010-01-29,-0.018932,4361.0,10.0600,4925,-0.037172,0.000013,43871.6600,44918.3000,-0.000656,1,-0.018945,-0.037185,0.000005
4,10001,2010-02-26,-0.000656,4361.0,10.0084,4925,0.034744,0.000061,43646.6324,43871.6600,0.020643,1,-0.000717,0.034683,0.000005
3,10001,2010-03-31,0.020643,4361.0,10.1700,4925,0.063668,0.000112,44351.3700,43646.6324,0.124385,1,0.020531,0.063556,0.000005
2,10001,2010-04-30,0.124385,6070.0,11.3900,4925,0.020036,0.000118,69137.3000,44351.3700,0.004829,1,0.124267,0.019918,0.000004
1,10001,2010-05-28,0.004829,6071.0,11.4000,4925,-0.079240,0.000114,69209.4000,69137.3000,-0.043421,1,0.004715,-0.079354,0.000007


In [21]:
data.to_parquet('stock_data.parquet')
merged_data.to_parquet('merged_stock_data.parquet')

---
# Betting against Beta strategy (BaB)

---
# Momentum Strategy (Mom)

---
# Idiosyncratic Volatility Strategy (IV)

---
# Optimal Fund Portfolio Return (STRAT)

---
# Performance and risk analysis for the Fund strategy

---
# Industry neutral strategy